In [1]:
plt.rcParams['figure.figsize'] = (15,5)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline

In [2]:
tr = pd.read_csv('train.csv', parse_dates=['Date'], low_memory=False)
te = pd.read_csv('test.csv', parse_dates=['Date'],low_memory=False)
st = pd.read_csv('store.csv')

# Competition

Fill in missing competition distance as 10 times more than existing max.

In [3]:
st[np.isnan(st.CompetitionDistance)]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
290,291,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
878,879,d,a,NaN,NaN,NaN,1,5,2013,"Feb,May,Aug,Nov"


In [4]:
st.loc[np.isnan(st.CompetitionDistance),'CompetitionDistance'] = st.CompetitionDistance.max()*10

In [5]:
full = pd.concat([tr,te], axis=0)
full = full.merge(st, on='Store').set_index('Date').sort_index()

In [6]:
full['CompetitionOpenSinceMonth'].fillna(1, inplace=True)
full['CompetitionOpenSinceYear'].fillna(1971, inplace=True)

def competition_open(df, year='CompetitionOpenSinceYear', month='CompetitionOpenSinceMonth'):
    open_ = pd.to_datetime(df[year].astype('int')*10000+
                           df[month].astype('int')*100+
                           15, format="%Y%m%d")
    return (open_ < df.index).astype('int')

for store in full['Store'].unique():
    full.ix[full['Store'] == store,'competition']= competition_open(full[full['Store']==store])

#full.drop(['CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth','Customers'], axis=1, inplace=True)
full.drop(['Customers'], axis=1, inplace=True)

# Promo2 Renew

In [7]:
full['PromoInterval'].fillna('Jan', inplace=True)
full['Promo2SinceYear'].fillna(2020, inplace=True)
full['Promo2SinceWeek'].fillna(1, inplace=True)

mon = full.index.map(lambda x: x.strftime('%b'))
per = full.PromoInterval.apply(lambda x: x.split(','))

full['Promo2_on'] = [x in y for x,y in zip(mon,per)]

def promo2_since(df, since_week='Promo2SinceWeek', since_year='Promo2SinceYear'):
    since_ = pd.to_datetime(df[since_year].astype('int')*1000+\
                           (df[since_week]*7).astype('int') - 6, format="%Y%j")
    return (since_ < df.index).astype('int')

for store in full['Store'].unique():
    full.ix[full['Store'] == store,'Promo2_R'] = \
    promo2_since(full[full['Store']==store])*full.ix[full['Store']==store, 'Promo2_on']

#full.drop(['Promo2SinceWeek', 'Promo2SinceYear','PromoInterval','Promo2_on'], axis=1, inplace=True)

full.head()

,DayOfWeek,Id,Open,Promo,Sales,SchoolHoliday,StateHoliday,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,competition,Promo2_on,Promo2_R
Date,,,,,,,,,,,,,,,,,,,,
2013-01-01,2,NaN,0,0,0,1,a,232,c,c,13570,5,2010,1,10,2013,"Mar,Jun,Sept,Dec",1,False,0
2013-01-01,2,NaN,0,0,0,1,a,180,d,a,5800,9,2010,0,1,2020,Jan,1,True,0
2013-01-01,2,NaN,0,0,0,1,a,405,a,a,1080,4,2008,1,14,2011,"Jan,Apr,Jul,Oct",1,True,1
2013-01-01,2,NaN,0,0,0,1,a,1022,a,c,1520,1,1971,0,1,2020,Jan,1,True,0
2013-01-01,2,NaN,0,0,0,1,a,1098,a,a,1830,11,2004,0,1,2020,Jan,1,True,0


# Add Tomorrow will be & Yesterday was a Holiday (TwH / YwH)

In [8]:
full.StateHoliday.fillna('0', inplace=True)

for store in full['Store'].unique():
    full.loc[full.Store==store, 'YwH'] = full.loc[full.Store==store,'StateHoliday'].shift(1)
    full.loc[full.Store==store, 'TwH'] = full.loc[full.Store==store,'StateHoliday'].shift(-1)

full.TwH.fillna('0', inplace=True)
full.YwH.fillna('0', inplace=True)

# Grouped dummies

In [9]:
full['Year'] = full.index.year
full['Month'] = full.index.month
full['WoY'] = full.index.weekofyear
full['Day'] = full.index.day
full['DoY'] = full.index.dayofyear
full['StateHoliday'] = np.unique(full.StateHoliday, return_inverse=True)[1]
full['StoreType'] = np.unique(full.StoreType, return_inverse=True)[1]
full['Assortment'] = np.unique(full.Assortment, return_inverse=True)[1]
full['TwH'] = np.unique(full.TwH, return_inverse=True)[1]
full['YwH'] = np.unique(full.YwH, return_inverse=True)[1]

In [10]:
full['SbM'] = np.log(full.groupby('Month')['Sales'].transform('mean'))
full['SbDoM'] = np.log(full.groupby('Day')['Sales'].transform('mean'))
full['SbDoW'] = np.log(full.groupby('DayOfWeek')['Sales'].transform('mean'))

In [11]:
full['SbMbS']   = np.log(full.groupby(['Store','Month'])['Sales'].transform('mean'))
full['SbDbS']   = np.log(full.groupby(['Store','Day'])['Sales'].transform('mean'))
full['SbDoWbS'] = np.log(full.groupby(['Store','DayOfWeek'])['Sales'].transform('mean'))

In [12]:
full.columns

Index(['DayOfWeek', 'Id', 'Open', 'Promo', 'Sales', 'SchoolHoliday',
       'StateHoliday', 'Store', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'competition', 'Promo2_on',
       'Promo2_R', 'YwH', 'TwH', 'Year', 'Month', 'WoY', 'Day', 'DoY', 'SbM',
       'SbDoM', 'SbDoW', 'SbMbS', 'SbDbS', 'SbDoWbS'],
      dtype='object')

# Find and remove "outliers"

In [13]:
indx = np.isnan(full['Id'])
train = full[indx]
test = full[~indx]

train = train[(train['Sales']!=0)&(train['Open']==1)]
train = train.drop(['Id'], axis=1)

test = test.drop('Sales', axis=1)

In [14]:
train.shape, test.shape

((844338, 32), (41088, 32))

In [15]:
preds = pd.read_csv('xgb_fill_5F_Strat.csv')

In [16]:
miss = ((preds['xgb_fill_5F_Strat'].values/train['Sales'].values-1)**2).argsort()[-100:]

np.sqrt(np.mean((np.delete(preds['xgb_fill_5F_Strat'].values, miss)/np.delete(train['Sales'].values,miss)-1)**2))

0.092763115672196805

In [17]:
idx = np.zeros(train.shape[0])
idx[miss] = 1
train = train.iloc[~idx.astype('bool'),]

In [18]:
train.shape, test.shape

((844238, 32), (41088, 32))

In [19]:
train.columns

Index(['DayOfWeek', 'Open', 'Promo', 'Sales', 'SchoolHoliday', 'StateHoliday',
       'Store', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'competition',
       'Promo2_on', 'Promo2_R', 'YwH', 'TwH', 'Year', 'Month', 'WoY', 'Day',
       'DoY', 'SbM', 'SbDoM', 'SbDoW', 'SbMbS', 'SbDbS', 'SbDoWbS'],
      dtype='object')

# Make Train / Test folds

In [20]:
import sklearn.cross_validation as cv

In [21]:
seed =1
rng = np.random.RandomState(seed)

In [22]:
idx = (train.Month==8)|(train.Month==9)
full_idx = np.arange(train.shape[0])
val_idx = full_idx[idx.values]
val_idx

array([195581, 195582, 195583, ..., 577450, 577451, 577452])

In [23]:
test_idx = []
for i, (_,tst) in enumerate(cv.KFold(len(val_idx), n_folds=5, random_state=rng)):
    test_idx.append(val_idx[tst])

In [24]:
train_idx = []
for i in range(5):
    train_idx.append(np.setdiff1d(full_idx,test_idx[i]))

In [25]:
for i in range(5):
    train_file='./train_b20_'+str(i)+'.csv'
    pd.DataFrame(train_idx[i], columns=['idx']).to_csv(train_file, index=False)
    fill_file='./fill_b20_'+str(i)+'.csv'
    pd.DataFrame(test_idx[i], columns=['idx']).to_csv(fill_file, index=False)

In [26]:
# tr = pd.read_csv('train_b20_0.csv')
# te = pd.read_csv('fill_b20_0.csv')

# Upload to R

In [27]:
train.to_csv('benchmark20_train.csv', index=False)

test.to_csv('benchmark20_test.csv', index=False)

In [28]:
train.columns

Index(['DayOfWeek', 'Open', 'Promo', 'Sales', 'SchoolHoliday', 'StateHoliday',
       'Store', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'competition',
       'Promo2_on', 'Promo2_R', 'YwH', 'TwH', 'Year', 'Month', 'WoY', 'Day',
       'DoY', 'SbM', 'SbDoM', 'SbDoW', 'SbMbS', 'SbDbS', 'SbDoWbS'],
      dtype='object')